# Approche Bayésienne

In [1]:
%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/Users/aurianeblarre/miniconda3/lib/python3.4/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
wine = pd.read_csv('Datasets/whiteWine.csv', delimiter = ";")

In [3]:
d = 11  #descripteurs
N =  wine.shape[0] #produits à tester
R = 5 #experts

y = np.array(wine.ix[:,d+2:]) #labels des annotateurs
x = np.array(wine.ix[:,0:d]) #variables explicatives
w0 = np.random.rand(1,d)

## Initialisation

In [11]:
def init_mu(y):
	mu = []
	for i in range(0,N):
		mu.append(np.sum(y[i])/R)
	return mu

mu0 = init_mu(y)

def sigma(z):
    return 1/(1+np.exp(-z))

def logit(z):
    return np.log(z/(1-z))

In [5]:
#Nos hypothèses
#Sensitivity
mean_prior1 = np.random.rand(R) #Quel pourcentage de bonnes réponses l'expert donne en moyenne ?
var_prior1 = np.random.rand(R) #Avec quelle incertitude ?
#Sensibility
mean_prior2 = np.random.rand(R) #Quel pourcentage de bonnes réponses l'expert donne en moyenne ?
var_prior2 = np.random.rand(R) #Avec quelle incertitude ?

#On en déduit les paramètres 
a_prior = np.random.rand(2,R) 
b_prior = np.random.rand(2,R)
for i in range(0,R):
    a_prior[0][i] = (-mean_prior1[i]**3 + mean_prior1[i]**2 -mean_prior1[i]*var_prior1[i]**2)/var_prior1[i]**2
    a_prior[1][i] = a_prior[0][i]*(1-mean_prior1[i])/mean_prior1[i]
    b_prior[0][i] = (-mean_prior2[i]**3 + mean_prior2[i]**2 -mean_prior2[i]*var_prior2[i]**2)/var_prior2[i]**2
    b_prior[1][i] = b_prior[0][i]*(1-mean_prior2[i])/mean_prior2[i]

gamma_prior = np.random.rand(d,d)
gamma_prior = np.linalg.inv(gamma_prior)

## Etape E

In [6]:
#E-step

def ai(alpha,y):
    a = []
    for i in range(0,N):
        proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
        a.append(proda)
    return a

def bi(beta,y):
    b = []
    for i in range(0,N):
        prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
        b.append(prodb)
    return b

def pi(x,w):
    p = []
    for i in range(0,N):
        p.append(sigma(x[i].dot(w.T)))
    return p

def mui(a,b,p):
    mu = []
    for i in range(0,N):
        mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
    return mu

def E_step(x,y,alpha,beta,w):
    a = ai(alpha,y)
    b = bi(beta,y)
    p = pi(x,w)
    mu = mui(a,b,p)
    return mu


## Maximum Log - Likelihood Estimator

In [7]:
def logLikelihood(w, y, alpha, beta):
    a = ai(alpha, y)
    b = bi(beta, y)
    p = pi(x, w)
    
    #On calcule directement la log-vraissemblance.
    vraissemblance = 0
    for i in range(0,N):
        vraissemblance = vraissemblance + np.log((a[i]*p[i])+b[i]*(1-p[i]))
        
    return vraissemblance

## Étape M

In [8]:
#M-step

def alpha_function(mu,y,a_prior):
	alpha = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += mu[i]*y[i][j]
			tmp2 += mu[i]
		alpha.append((a_prior[0][j]-1+tmp1)/(a_prior[0][j]+a_prior[1][j]-2+tmp2))
	return alpha

def beta_function(mu,y,b_prior):
	beta = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += (1-mu[i])*(1-y[i][j])
			tmp2 += 1-mu[i]
		beta.append((b_prior[0][j]-1+tmp1)/(b_prior[0][j]+b_prior[1][j]-2+tmp2))
	return beta

def updateW(w,x, eta, mu, gamma_prior):
    g = 0
    for i in range(0,N):
        g += (mu[i] - sigma(x[i].dot(w.T)))*x[i]
    tmp = np.reshape(-gamma_prior.dot(w.T),g.shape) # (11,1) -> (11,)
    g += tmp
    
    H = np.zeros((d,d))
    for i in range(0,N):
        H -= sigma(x[i].dot(w.T))*(1-sigma(x[i].dot(w.T)))*((x[i].reshape(11,1))*(x[i].reshape(1,11)))
    H -= gamma_prior
    w = w - eta*np.linalg.inv(H).dot(g)
    return w

In [9]:
updateW(w0,x,0.1,mu0,gamma_prior)

array([[-35334.92901159, -28871.41002449, -37022.64471383, -24159.64331136,
        -33242.01854823, -20056.53651041, -34332.7256707 ,  -6830.87748493,
        -27080.53161824, -13806.67227696, -20106.51189462]])

## Itérations

In [10]:
mu = init_mu(y)
alpha = alpha_function(mu,y,a_prior)
beta = beta_function(mu,y,b_prior)
diff_w = 10
w = w0

compteur = 0

#while (np.linalg.norm(diff_w) > 0.001) : # Limite de convergence à decider
while (compteur < 100):
    mu = E_step(x,y,alpha,beta,w)
    alpha = alpha_function(mu,y,a_prior)
    beta = beta_function(mu,y,b_prior)
    w_bis = updateW(w,x,0.01,mu,gamma_prior)
    diff_w = w - w_bis
    w = w_bis
    if (compteur % 10 == 0):
        print ("ITERATION : ", compteur)
        print("Vraissemblance : ", logLikelihood(w, y, alpha, beta))
        print("Norme de diff_w : ", np.linalg.norm(diff_w))
        print("Alpha : ", alpha)
        print("Beta : ", beta)
    compteur = compteur + 1

ITERATION :  0
Vraissemblance :  [-12941.84181545]
Norme de diff_w :  0.0189439655636
Alpha :  [array([ 0.89419982]), array([ 0.70627847]), array([ 0.51154001]), array([ 0.40361302]), array([ 0.1026994])]
Beta :  [array([ 0.4933009]), array([ 0.39185742]), array([ 0.55288447]), array([ 0.55593145]), array([ 0.84461565])]
ITERATION :  10
Vraissemblance :  [-12941.84181559]
Norme de diff_w :  0.0171301718401
Alpha :  [array([ 0.89419982]), array([ 0.70627847]), array([ 0.51154001]), array([ 0.40361302]), array([ 0.1026994])]
Beta :  [array([ 0.49330092]), array([ 0.39185747]), array([ 0.55288447]), array([ 0.55593144]), array([ 0.84461576])]
ITERATION :  20
Vraissemblance :  [-12941.84181615]
Norme de diff_w :  0.0154828695169
Alpha :  [array([ 0.89419982]), array([ 0.70627847]), array([ 0.51154001]), array([ 0.40361302]), array([ 0.1026994])]
Beta :  [array([ 0.49330103]), array([ 0.39185767]), array([ 0.55288445]), array([ 0.55593141]), array([ 0.84461627])]
ITERATION :  30
Vraissembla

In [29]:
#weight of each expert
weights = logit(np.array(alpha))+logit(np.array(beta))

#Sensitivity and sensibility of classifiers
alpha_clas = []
beta_clas = []
for j in range(0,d):
    tmp = 0
    tmp2 = 0
    for i in range(0,N):
        tmp += mu[j]*x[i][j]
        tmp2 += (1-mu[j])*(1-x[i][j])
    alpha_clas.append(tmp/np.sum(mu))
    beta_clas.append(tmp2/(N-np.sum(mu)))
alpha_clas = np.array(alpha_clas)
beta_clas = np.array(beta_clas)


[[  6.85478903e+00]
 [  2.78241174e-01]
 [  3.34191573e-01]
 [  6.39141613e+00]
 [  4.57723652e-02]
 [  3.53080920e+01]
 [  1.38360685e+02]
 [  9.94027574e-01]
 [  3.18826727e+00]
 [  4.89846974e-01]
 [  1.05142691e+01]] [[  0.00000000e+00]
 [  0.00000000e+00]
 [  1.85919375e-09]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [ -2.10763051e-07]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [ -3.53501284e-03]]


In [ ]:
class model:
    """Classe qui encapsule l'apprentissage"""
    
    def __init__(self, maxIter = 10000, eta = 0.01):
        """Constructeur"""
        self.maxIter = maxIter
        self.eta = eta
        
    def initMu(self, y):
        """Initialisation de mu"""
        mu = []
        for i in range(0,N):
            mu.append(np.sum(y[i])/R)
        return mu
    
    def ai(self, alpha, y):
        """Update du vecteur a (1xN)"""
        a = []
        for i in range(0,N):
            proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
            a.append(proda)
        self.a = a
        
    def bi(self, beta, y):
        """Update du vecteur b (1xN)"""
        b = []
        for i in range(0,N):
            prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
            b.append(prodb)
        self.b = b
        
    def pi(self, x, w):
        """Update du vecteur p (1xN)"""
        p = []
        for i in range(0,N):
            p.append(sigma(x[i].dot(w.T)))
        self.p = p
        
    def mui(self, a,b,p):
        """Update de"""
        mu = []
        for i in range(0,N):
            mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
        self.mu = mu
    
    def EStep(x,y,alpha,beta,w):
        CE = 0 #Conditionnal excepectation
        a = ai(alpha,y)
        b = bi(beta,y)
        p = pi(x,w)
        mu = mui(a,b,p)
        for i in range(0,N):
            CE += mu[i]*np.log(p[i])*a[i]+(1-mu[i])*np.log(1-p[i])*b[i]
        